# Assignment 6: Analysis of accuracy decrease of tracking in conditions of biased state noise

## Team №6:
1. Angelina Prokopeva
2. Nikita Gorbadey
3. Mark Griguletskii
4. Stanislav Savushkin

03.10.2019, Skoltech

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from IPython.display import Image
import os

if not os.path.exists("images"):
    os.mkdir("images")

In [54]:
generate_report = False
def plot(trace_num, x_data, y_data, xlable = 'xlable', ylable = 'ylable',
                legend = 'legend', title = 'title', mode='lines'):
    plot.counter += 1
    fig_name = 'images/' + str(plot.counter) + '.jpg'
    fig = go.Figure()
    for i in range(trace_num):
        fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode=mode, name=legend[i], showlegend = True))
    fig.update_layout(
        title=go.layout.Title(
            text=title,
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=xlable
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text=ylable
            )
        )
    )
    if generate_report is True:
        fig.write_image(fig_name)
        display(Image(fig_name))
    else:
        fig.show()
plot.counter = 0

### Working progress

In [55]:
def motion_tragectory(size, T, mean_a, sigma_a, X_init=0, V_init=0):
    "Function for generating motion trajectory with normally distributed random acceleration"
    out = np.zeros(size)
    V = np.zeros(size)
    out[0] = X_init
    V[0] = V_init
    a = np.random.normal(loc=mean_a, scale=sigma_a, size=size)
    for i in range(1, size):
        V[i] = V[i-1] + a[i-1]*T
        out[i] = out[i-1] + V[i-1]*T + (a[i-1]*T**2)/2
    return out

In [56]:
def measured_trajectory(data, sigma):
    "Supplementary function for generating measured trajectory"
    return np.add(data, np.random.normal(loc=0, scale=sigma, size=data.size))

In [57]:
N = 200
X_init = 5
V_init = 1
T = 1
mean_a=0.2
sigma_a = 0.2
sigma_n = 20

# Generating true trajectory
x = motion_tragectory(N, T, sigma_a, mean_a, X_init=X_init, V_init=V_init)

# Generating measurements
z = measured_trajectory(x, sigma_n)
num_points = np.linspace(1, N, num=N)

In [58]:
plot(2, [num_points, num_points], [x, z], title='True trajectory and its measurements', xlable='steps', ylable='coordinate',
     legend=['real trajectory', 'measured trajectory'])

In [65]:
def kalman_filter(z, T, X_0=[0, 0], P_0=[[100, 0], [0, 100]], sigma_a=1, sigma_n=1, shift=7):
    shape = (z.size, 2)
    X_pred = np.zeros(shape)
    X_filt = np.zeros(shape)
    x_i_pred = np.zeros(shape[0])
    F = np.array([[1, T], [0, 1]])
    G = np.array([T**2/2, T]).T
    H = np.array([1, 0])
    X_0 = [2, 0]
    P_0_0 = [[10000, 0], [0, 10000]]
    
    # prepare parameters
    Q = G.dot(G.T.dot(sigma_a**2))
    R = sigma_n**2
    P_pred = np.zeros((shape[0], 2, 2))
    P_filt = np.zeros((shape[0], 2, 2))
    K = np.zeros(shape)

    X_filt[0] = X_0
    P_filt[0] = P_0
    K[0] = P_filt[0].dot(H.T)/(H.dot(P_filt[0]).dot(H.T) + R)

    for i in range(1, shape[0]):
        # Make prediction
        X_pred[i] = F.dot(X_filt[i-1])
        P_pred[i] = np.dot(F, np.dot(P_filt[i-1], F.T)) + Q

        # Make filtration
        K[i] = P_pred[i].dot(H.T)/(H.dot(P_pred[i]).dot(H.T) + R)
        X_filt[i] = X_pred[i] + K[i]*(z[i] - H.dot(X_pred[i]))
        P_filt[i] = (np.eye(shape[1]) - np.outer(K[i], H)).dot(P_pred[i])
        
    x_i_pred = np.roll(np.linalg.matrix_power(F, shift).dot(X_filt.T)[0], shift)
    x_i_pred[:shift] = X_0[0]

    return X_filt, X_pred, K, P_filt, x_i_pred

In [66]:
kalman_data = kalman_filter(z, T=1, X_0=[2, 0], P_0=[[10000, 0], [0, 10000]], sigma_a=sigma_a, sigma_n=sigma_n)

In [67]:
plot(3, [num_points, num_points, num_points], [x, z, kalman_data[0][:,0]], title='Kalman filter visualization', xlable='steps', ylable='coordinate',
     legend=['real trajectory', 'measured trajectory', 'Kalman filter'])

In [71]:
def error_calculation(runs=500, steps=200, T=1, mean_a=0.2, sigma_a=0.2, X_init=0, V_init=0,
                         X_0=[2, 0], P_0=[[10000, 0], [0, 10000]], sigma_n=20, shift=7, sigma_a_filter=0.2):
    err_filt = np.zeros(steps)
    err_filt_pred = np.zeros(steps-shift)
    
    for i in range(runs):
        # Generating true trajectory
        x = motion_tragectory(steps, T, mean_a=mean_a, sigma_a=sigma_a, X_init=X_init, V_init=V_init)
        
        # Generating measurements
        z = measured_trajectory(x, sigma_n)
        
        # Perform Kalman filter
        X_filt, X_pred, K, P_filt, x_i_pred = kalman_filter(z, T=T, X_0=X_0, P_0=P_0, 
                                                            sigma_a=sigma_a_filter, sigma_n=sigma_n, shift=shift)
        
        # Calculate errors starting from 3rd step
        err_filt += (x - X_filt[:,0])**2
        err_filt_pred += (x_i_pred[shift:] - x[shift:])**2
    
    # Take square root for each step
    err_filt = np.sqrt(err_filt / (runs - 1))
    err_filt_pred = np.sqrt(err_filt_pred / (runs - 1))
    
    return err_filt, err_filt_pred

In [75]:
M = 500
N = 200
steps = np.linspace(2, N-2)
err_filt, err_filt_pred = error_calculation(P_0=[[10000, 0], [0, 10000]])
kalman_data = kalman_filter(z, T=1, X_0=[2, 0], P_0=[[10000, 0], [0, 10000]], sigma_a=sigma_a, sigma_n=sigma_n)
filt_err = np.sqrt(kalman_data[3][:,0,0])

In [78]:
plot(2, [steps, steps], [err_filt[2:], filt_err[2:]], title='Average error evolution',
     xlable='step', ylable='value',
     legend=['true estim err', 'filter err cov matrix'])

In [79]:
def kalman_filter_with_bias(z, T, X_0=[0, 0], P_0=[[100, 0], [0, 100]], sigma_a=1, sigma_n=1, shift=7, q=0.2):
    shape = (z.size, 2)
    X_pred = np.zeros(shape)
    X_filt = np.zeros(shape)
    x_i_pred = np.zeros(shape[0])
    F = np.array([[1, T], [0, 1]])
    G = np.array([T**2/2, T]).T
    H = np.array([1, 0])
    X_0 = [2, 0]
    P_0_0 = [[10000, 0], [0, 10000]]
    
    # prepare parameters
    Q = G.dot(G.T.dot(sigma_a**2))
    R = sigma_n**2
    P_pred = np.zeros((shape[0], 2, 2))
    P_filt = np.zeros((shape[0], 2, 2))
    K = np.zeros(shape)

    X_filt[0] = X_0
    P_filt[0] = P_0
    K[0] = P_filt[0].dot(H.T)/(H.dot(P_filt[0]).dot(H.T) + R)

    for i in range(1, shape[0]):
        # Make prediction
        X_pred[i] = F.dot(X_filt[i-1]) + G.dot(q)
        P_pred[i] = np.dot(F, np.dot(P_filt[i-1], F.T)) + Q

        # Make filtration
        K[i] = P_pred[i].dot(H.T)/(H.dot(P_pred[i]).dot(H.T) + R)
        X_filt[i] = X_pred[i] + K[i]*(z[i] - H.dot(X_pred[i]))
        P_filt[i] = (np.eye(shape[1]) - np.outer(K[i], H)).dot(P_pred[i])
        
    x_i_pred = np.roll(np.linalg.matrix_power(F, shift).dot(X_filt.T)[0], shift)
    x_i_pred[:shift] = X_0[0]

    return X_filt, X_pred, K, P_filt, x_i_pred

In [80]:
kalman_data = kalman_filter(z, T=1, X_0=[2, 0], P_0=[[10000, 0], [0, 10000]], sigma_a=sigma_a, sigma_n=sigma_n)

In [81]:
plot(3, [num_points, num_points, num_points], [x, z, kalman_data[0][:,0]], title='Kalman filter visualization', xlable='steps', ylable='coordinate',
     legend=['real trajectory', 'measured trajectory', 'Kalman filter'])

In [82]:
def error_calculation_with_bias(runs=500, steps=200, T=1, mean_a=0.2, sigma_a=0.2, X_init=0, V_init=0,
                         X_0=[2, 0], P_0=[[10000, 0], [0, 10000]], sigma_n=20, shift=7, sigma_a_filter=0.2):
    err_filt = np.zeros(steps)
    err_filt_pred = np.zeros(steps-shift)
    
    for i in range(runs):
        # Generating true trajectory
        x = motion_tragectory(steps, T, mean_a=mean_a, sigma_a=sigma_a, X_init=X_init, V_init=V_init)
        
        # Generating measurements
        z = measured_trajectory(x, sigma_n)
        
        # Perform Kalman filter
        X_filt, X_pred, K, P_filt, x_i_pred = kalman_filter_with_bias(z, T=T, X_0=X_0, P_0=P_0, q=mean_a,
                                                            sigma_a=sigma_a_filter, sigma_n=sigma_n, shift=shift)
        
        # Calculate errors starting from 3rd step
        err_filt += (x - X_filt[:,0])**2
        err_filt_pred += (x_i_pred[shift:] - x[shift:])**2
    
    # Take square root for each step
    err_filt = np.sqrt(err_filt / (runs - 1))
    err_filt_pred = np.sqrt(err_filt_pred / (runs - 1))
    
    return err_filt, err_filt_pred

In [83]:
M = 500
N = 200
steps = np.linspace(2, N-2)
err_filt, err_filt_pred = error_calculation_with_bias(P_0=[[10000, 0], [0, 10000]])
kalman_data = kalman_filter(z, T=1, X_0=[2, 0], P_0=[[10000, 0], [0, 10000]], sigma_a=sigma_a, sigma_n=sigma_n)
filt_err = np.sqrt(kalman_data[3][:,0,0])

In [84]:
plot(2, [steps, steps], [err_filt[2:], filt_err[2:]], title='Average error evolution',
     xlable='step', ylable='value',
     legend=['true estim err', 'filter err cov matrix'])